In [84]:
import pandas as pd
import numpy as np
import random
from nltk.corpus import wordnet
import re
import csv
from tqdm import tqdm_notebook as tqdm
# from nltk.corpus import stopwords as stop_words

import pandas as pd
import numpy as np
import string

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/IRE_Major_Project/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
            'ours', 'ourselves', 'you', 'your', 'yours', 
            'yourself', 'yourselves', 'he', 'him', 'his', 
            'himself', 'she', 'her', 'hers', 'herself', 
            'it', 'its', 'itself', 'they', 'them', 'their', 
            'theirs', 'themselves', 'what', 'which', 'who', 
            'whom', 'this', 'that', 'these', 'those', 'am', 
            'is', 'are', 'was', 'were', 'be', 'been', 'being', 
            'have', 'has', 'had', 'having', 'do', 'does', 'did',
            'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
            'because', 'as', 'until', 'while', 'of', 'at', 
            'by', 'for', 'with', 'about', 'against', 'between',
            'into', 'through', 'during', 'before', 'after', 
            'above', 'below', 'to', 'from', 'up', 'down', 'in',
            'out', 'on', 'off', 'over', 'under', 'again', 
            'further', 'then', 'once', 'here', 'there', 'when', 
            'where', 'why', 'how', 'all', 'any', 'both', 'each', 
            'few', 'more', 'most', 'other', 'some', 'such', 'no', 
            'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
            'very', 's', 't', 'can', 'will', 'just', 'don', 
'should', 'now', '']

In [0]:
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word): 
        for l in syn.lemmas(): 
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym) 
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)


In [0]:
def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            #print("replaced", random_word, "with", synonym)
            num_replaced += 1
        if num_replaced >= n: #only replace up to n words
            break

    #this is stupid but we need it, trust me
    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')

    return new_words


In [0]:
def random_deletion(words, p):

    #obviously, if there's only one word, don't delete it
    if len(words) == 1:
        return words

    #randomly delete words with probability p
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    #if you end up deleting all words, just return a random word
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return new_words

In [0]:
def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
    return new_words

In [0]:
def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    return new_words

In [0]:
def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)


In [0]:
def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    return new_words

In [0]:
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line


In [0]:
def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    sentence = get_only_chars(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word is not '']
    num_words = len(words)
    
    augmented_sentences = []
    num_new_per_technique = int(num_aug/4)+1
    n_sr = max(1, int(alpha_sr*num_words))
    n_ri = max(1, int(alpha_ri*num_words))
    n_rs = max(1, int(alpha_rs*num_words))

    #sr
    for _ in range(num_new_per_technique):
        a_words = synonym_replacement(words, n_sr)
        augmented_sentences.append(' '.join(a_words))

    #ri
    for _ in range(num_new_per_technique):
        a_words = random_insertion(words, n_ri)
        augmented_sentences.append(' '.join(a_words))

    #rs
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(' '.join(a_words))

    #rd
    for _ in range(num_new_per_technique):
        a_words = random_deletion(words, p_rd)
        augmented_sentences.append(' '.join(a_words))

    augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
    random.shuffle(augmented_sentences)

    #trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    #append the original sentence
    augmented_sentences.append(sentence)

    return augmented_sentences

In [0]:
data = pd.read_csv(root_path+'preprocessed.csv')
data.head()

from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.3)

train.to_csv(root_path+'dl_train.csv', index=False)
test.to_csv(root_path+'dl_test.csv', index=False)

In [70]:
data = pd.read_csv(root_path+'dl_train.csv')
data.head()
mcols = list(data.columns)
print(mcols)

['id', 'tweet', 'new_tweet', 'user_mentions', 'n_hash_tags', 'n_urls', 'n_emojis', 'subtask_a', 'subtask_b', 'subtask_c', 'new_tweet_length', 'original_tweet_length']


In [34]:
data['subtask_c'].value_counts()

IND    1715
GRP     748
OTH     264
Name: subtask_c, dtype: int64

In [0]:
def write_in_file(file_writer, row, new_text):
    c1 = row['id']
    c2 = row['tweet']
    c3 = new_text 	 	
    c4 = row['user_mentions'] 	
    c5 = row['n_hash_tags']
    c6 = row['n_urls']
    c7 = row['n_emojis']
    c8 = row['subtask_a']
    c9 = row['subtask_b']
    c10 = row['subtask_c'] 	 	 	 	
    c11 = len(new_text.split())
    c12 = row['original_tweet_length']

    file_writer.writerow([c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12])
    return

In [0]:
def augment_data(num_aug, new_filename,n):
    
    given_data = pd.read_csv(root_path+'dl_train.csv', error_bad_lines=False, names=n, skiprows=1)
    n = list(given_data.columns)
    
    
    with open(new_filename, 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter=',')
        tsv_writer.writerow(n)
        
        for index, row in tqdm(given_data.iterrows()):
            write_in_file(tsv_writer, row, row['new_tweet'])
            try:
              augmented_data = eda(row['new_tweet'], num_aug=num_aug)
              for data in augmented_data:
                  write_in_file(tsv_writer, row, data)
            except:
              pass

In [93]:
for degree in range(3):
  augment_data(degree,root_path+'dl_augmented_train.csv',mcols)    

In [0]:
df = pd.read_csv(root_path+'dl_augmented_train.csv')

In [94]:
df.head()

,id,tweet,new_tweet,user_mentions,n_hash_tags,n_urls,n_emojis,subtask_a,subtask_b,subtask_c,new_tweet_length,original_tweet_length
0,10305,@USER I definitely believe that she is credibl...,definitely believe credible compare man lie oa...,1,0,0,0,NOT,NaN,NaN,55,18
1,10305,@USER I definitely believe that she is credibl...,definitely believe credible compare man lie oa...,1,0,0,0,NOT,NaN,NaN,53,18
2,10305,@USER I definitely believe that she is credibl...,definitely man credible compare believe lie oa...,1,0,0,0,NOT,NaN,NaN,53,18
3,10305,@USER I definitely believe that she is credibl...,definitely believe credible compare man lie oa...,1,0,0,0,NOT,NaN,NaN,53,18
4,10106,@USER Still coverup. And bet the anonymous wr...,still coverup bet anonymous writer op ed ny ti...,1,3,0,0,OFF,TIN,IND,167,46
